# Curate entity identifiers

To make data queryable by an entity identifier, one needs to ensure that identifiers comply to a chosen standard.
Bionty enables this by curating data against the versionized ontologies using {meth}`~bionty.Entity.curate`.

We'll demonstrate this by first curating genes and second CellMarkers where not all values can be immediately mapped.

Let's start by importing the required modules from Bionty and Pandas.

In [ ]:
from bionty import Gene, CellMarker, lookup
import pandas as pd

## Curating genes

To illustrate it, let us generate a DataFrame that stores a number of gene identifiers, some of which corrupted.

In [ ]:
data = {
    "gene symbol": ["A1CF", "A1BG", "FANCD1", "corrupted"],
    "hgnc id": ["HGNC:24086", "HGNC:5", "HGNC:1101", "corrupted"],
    "ensembl_gene_id": [
        "ENSG00000148584",
        "ENSG00000121410",
        "ENSG00000188389",
        "corrupted",
    ],
}
df_orig = pd.DataFrame(data).set_index("ensembl_gene_id")

In [ ]:
df_orig

We require a reference identifier (specified as the `reference_id` parameter for curate).
The list can be looked up using {meth}`~bionty.lookup`.
Examples are "ontology_id", which corresponds to the IDs of the ontology terms (e.g. 'ENSG00000148584')
or "name" which corresponds to the ontology term names (e.g. 'A1CF').

In [ ]:
lookup.gene_id

To curate the DataFrame into queryable form, we create an index that corresponds to a default identifier.
By default we use `ensembl_gene_id`. The default behavior is to curate the index if a column name is not provided.

First we create a `Gene()` instance using the default source database and version.

In [ ]:
gene = Gene()

First we can check whether any of our values are mappable against the ontology.

In [ ]:
gene.inspect(df_orig.index, reference_id=gene.ensembl_gene_id)

We have identified 3 terms that are mappable against the Ontology. Let's curate them by mapping them against the ontology. By default, Bionty uses the index column if not specified otherwise.

In [ ]:
gene.curate(df_orig)

The curated DataFrame has now been reindexed by the curated cell types.
A new column `orig_index` containing the original index has been added.
Furthermore, a new column `__curated__` containing booleans of whether the data could be successfully curated or not has been added.

The same procedure is available for gene symbols. First, we inspect which symbols are mappable against the ontology.

In [ ]:
gene.inspect(df_orig["gene symbol"], reference_id=gene.symbol)

Apparently 2 of the gene symbols are mappable. Bionty further warns us that some of our symbols can be mapped into standardized symbols.

In [ ]:
mapped_symbol_synonyms = gene.map_synonyms(
    df_orig["gene symbol"], reference_id=gene.symbol
)
mapped_symbol_synonyms

We can store them in our DataFrame further use.

In [ ]:
df_orig["non-synonymous gene symbol"] = mapped_symbol_synonyms

You may provide a column name to curate a specific column against a reference identifier.
When mapping symbols, the function will automatically convert the aliases into standardized symbols.
In this example, `FANCD1` is converted into `BRACA2`.

In [ ]:
gene.curate(df_orig, column="gene symbol", reference_id=lookup.gene_id.symbol)

This is synonymous to:

In [ ]:
gene.curate(df_orig, column="gene symbol", reference_id=gene.symbol)

Of course this also works with other columns such as "hgnc id".

In [ ]:
gene.curate(df_orig, column="hgnc id", reference_id=lookup.gene_id.hgnc_id)

## Match (unmappable) cell markers to the reference

Depending on how the data was collected and which terminology was used, it is not always possible to curate the values.
Some values might have used a different standard or are simply corrupted.

This section will demonstrate how to look up unmatched terms and curating them using The CellMarker entity.
First, we create an example Pandas DataFrame containing a few valid and invalid cell markers (antibody targets) and features (Time) from a flow cytometry dataset.

In [ ]:
markers = pd.DataFrame(
    index=[
        "KI67",
        "CCR7x",
        "CD14",
        "CD8",
        "CD45RA",
        "CD4",
        "CD3",
        "CD127",
        "PD1",
        "Invalid-1",
        "Invalid-2",
        "CD66b",
        "Siglec8",
        "Time",
    ]
)

Let's instantiate the CellMarker ontology with the default database and version.

In [ ]:
cell_marker = CellMarker()

First, we can have a look at the cell marker table that we just loaded.

In [ ]:
df = cell_marker.df()

In [ ]:
df.head()

Now let’s check which cell markers from the file can be found in the reference.
We do this using the `.curate function`:

In [ ]:
cell_marker.curate(markers)

From the logging, it can be seen that 4 terms were not found in the reference!

Among them `Time`, `Invalid-1` and `Invalid-2` are non-marker channels which won’t be curated by cell marker.

Note, certain markers will be converted to synonyms such as `PD1` -> `PD-1`.

We don't really find `CCR7x`, let's check in the lookup with auto-completion:

In [ ]:
cell_marker_lookup = cell_marker.lookup()

```{figure} ../img/lookup_ccr7.png
---
width: 70%
align: left
class: with-shadow
---
```

In [ ]:
cell_marker_lookup.CCR7

Indeed we find it should be CCR7, we had a typo there with `CCR7x`.

Now let’s fix the markers so all of them can be linked:

```{tip}
Using the .lookup instead of passing a string helps eliminate possible typos!
```

In [ ]:
curated_df = markers.rename(index={"CCR7x": cell_marker_lookup.CCR7.name})

OK, now we can try to run curate again and all cell markers are linked!

In [ ]:
cell_marker.curate(curated_df)